###Kaggle Competition Link:
https://www.kaggle.com/c/plant-seedlings-classification

###Approach:

We will try training a ResNet50 model with about 10% of the layers unfrozen.

###Mounting Google Drive

In [1]:
#Access the dataset from Google Drive (drive will be mounted to : "/content/gdrive/My Drive")
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


###Reading the dataset which is saved as a zip file on google drive

In [2]:
from zipfile import ZipFile
dataset = ZipFile('/content/gdrive/My Drive/Python Notebooks/Datasets/Image Datasets/Plant Seedlings Classification.zip')

In [3]:
#extract data
dataset.extractall(path='')

In [4]:
!ls -l train

total 208
drwxr-xr-x 2 root root 12288 Dec 21 09:14  Black-grass
drwxr-xr-x 2 root root 20480 Dec 21 09:14  Charlock
drwxr-xr-x 2 root root 12288 Dec 21 09:14  Cleavers
drwxr-xr-x 2 root root 20480 Dec 21 09:14 'Common Chickweed'
drwxr-xr-x 2 root root 12288 Dec 21 09:14 'Common wheat'
drwxr-xr-x 2 root root 20480 Dec 21 09:14 'Fat Hen'
drwxr-xr-x 2 root root 28672 Dec 21 09:14 'Loose Silky-bent'
drwxr-xr-x 2 root root 12288 Dec 21 09:15  Maize
drwxr-xr-x 2 root root 20480 Dec 21 09:15 'Scentless Mayweed'
drwxr-xr-x 2 root root 12288 Dec 21 09:15 'Shepherds Purse'
drwxr-xr-x 2 root root 20480 Dec 21 09:15 'Small-flowered Cranesbill'
drwxr-xr-x 2 root root 20480 Dec 21 09:15 'Sugar beet'


###Building Batch Generators for training and validation sets

In [5]:
import tensorflow as tf
import numpy as np

#Define image parameters
img_size = 224
img_depth = 3

In [6]:
#Function to normalize image for a ResNet50 model
def normalize_data(img):
  return tf.keras.applications.resnet50.preprocess_input(img)

In [7]:
#Define Training Data Generator with Augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,
                                                          width_shift_range=0.2,
                                                          height_shift_range=0.2,
                                                          horizontal_flip=True,
                                                          preprocessing_function = normalize_data,
                                                          validation_split=0.3)

In [8]:
train_generator = datagen.flow_from_directory('train',
                                              target_size = (img_size, img_size),
                                              subset = 'training',
                                              batch_size=32)

val_generator = datagen.flow_from_directory('train',
                                            target_size = (img_size, img_size),
                                            subset = 'validation',
                                            batch_size=32)

Found 3330 images belonging to 12 classes.
Found 1420 images belonging to 12 classes.


In [9]:
#Checking shape of features (images) and Labels (class) returned by ImageDataGenerator
X, y = next(train_generator)

print('Input features shape', X.shape)
print('Actual labels shape', y.shape)

Input features shape (32, 224, 224, 3)
Actual labels shape (32, 12)


###Load Pre-Trained Model 

In [10]:
#Clear old model from memory
tf.keras.backend.clear_session()

#Call Model
model = tf.keras.applications.ResNet50(include_top=False, #Do not include classification layer for imagenet
                                       input_shape=(img_size,img_size,img_depth),
                                       weights='imagenet')

94765736/94765736 [==============================] - 5s 0us/step


In [11]:
len(model.layers)

175

In [12]:
#Set pre-trained model layers to not trainable/frozen
for layer in model.layers:
    layer.trainable = False

In [13]:
#Unfreezing all layers after layer 158
for layer in model.layers[158:]:
    layer.trainable = True    

In [14]:
#Check if layers have been frozen
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

###Adding Output Layer for new classes

In [15]:
#Get output layer of the pre-trained model
model_op = model.output

#Add global average pool to reduce number of features and flatten the output
map = tf.keras.layers.GlobalAveragePooling2D()(model_op)

#Add our own final output layer
prediction = tf.keras.layers.Dense(12,activation='softmax')(map)

###Building Final Model for Classification

In [16]:
#Using Keras Model class
final_model = tf.keras.models.Model(inputs=model.input, #Pre-trained model input as input layer
                                    outputs=prediction) #Output layer added

In [17]:
#Compile final model
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
#Check final model summary
final_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [19]:
#Add checkpoint for saving best model
model_checkpoint=tf.keras.callbacks.ModelCheckpoint('plant_seedlings_resnet50.h5', 
                                                    save_best_only=True, 
                                                    monitor='val_accuracy', 
                                                    mode='max', 
                                                    verbose=1)

#Add early stopping to avoid wasted epochs
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                                                  min_delta = 0.01,
                                                  patience = 3,
                                                  verbose = 1,
                                                  mode = 'auto',
                                                  restore_best_weights = True)

###Beginning Training

In [20]:
final_model.fit(train_generator,
                epochs=100,
                steps_per_epoch= 3330//32,
                validation_data=val_generator,
                validation_steps = 1420//32,
                callbacks=[model_checkpoint])

Epoch 1/100
104/104 [==============================] - ETA: 0s - loss: 0.7286 - accuracy: 0.7690
Epoch 1: val_accuracy improved from -inf to 0.77060, saving model to plant_seedlings_resnet50.h5
104/104 [==============================] - 116s 1s/step - loss: 0.7286 - accuracy: 0.7690 - val_loss: 0.6701 - val_accuracy: 0.7706
Epoch 2/100
104/104 [==============================] - ETA: 0s - loss: 0.3467 - accuracy: 0.8799
Epoch 2: val_accuracy improved from 0.77060 to 0.83381, saving model to plant_seedlings_resnet50.h5
104/104 [==============================] - 100s 965ms/step - loss: 0.3467 - accuracy: 0.8799 - val_loss: 0.6048 - val_accuracy: 0.8338
Epoch 3/100
104/104 [==============================] - ETA: 0s - loss: 0.2745 - accuracy: 0.8996
Epoch 3: val_accuracy improved from 0.83381 to 0.85369, saving model to plant_seedlings_resnet50.h5
104/104 [==============================] - 101s 975ms/step - loss: 0.2745 - accuracy: 0.8996 - val_loss: 0.4992 - val_accuracy: 0.8537
Epoch 4/10

KeyboardInterrupt: ignored

###Lowering Learning Rate and continuing training

In [21]:
#Load saved weights
final_model.load_weights('/content/plant_seedlings_resnet50.h5')

In [22]:
#Re-compile final model with learning rate lowered from 0.001 to 0.0001
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
final_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
final_model.fit(train_generator,
                epochs=100,
                steps_per_epoch= 3803//64,
                validation_data=val_generator,
                validation_steps = 947//64,
                callbacks=[model_checkpoint,early_stopping])

Epoch 1/100
59/59 [==============================] - ETA: 0s - loss: 0.0389 - accuracy: 0.9865
Epoch 1: val_accuracy improved from 0.93821 to 0.93892, saving model to plant_seedlings_resnet50.h5
59/59 [==============================] - 73s 1s/step - loss: 0.0389 - accuracy: 0.9865 - val_loss: 0.2208 - val_accuracy: 0.9389
Epoch 2/100
59/59 [==============================] - ETA: 0s - loss: 0.0280 - accuracy: 0.9889
Epoch 2: val_accuracy improved from 0.93892 to 0.94531, saving model to plant_seedlings_resnet50.h5
59/59 [==============================] - 71s 1s/step - loss: 0.0280 - accuracy: 0.9889 - val_loss: 0.2005 - val_accuracy: 0.9453
Epoch 3/100
59/59 [==============================] - ETA: 0s - loss: 0.0239 - accuracy: 0.9915
Epoch 3: val_accuracy did not improve from 0.94531
59/59 [==============================] - 68s 1s/step - loss: 0.0239 - accuracy: 0.9915 - val_loss: 0.2418 - val_accuracy: 0.9389
Epoch 4/100
 8/59 [===>..........................] - ETA: 34s - loss: 0.0133 

KeyboardInterrupt: ignored

In [24]:
#Load saved weights
final_model.load_weights('/content/plant_seedlings_resnet50.h5')

In [25]:
#Re-compile final model with learning rate lowered from 0.0001 to 0.00001
opt = tf.keras.optimizers.Adam(learning_rate=0.00001)
final_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
#Continue training 
final_model.fit(train_generator,
                epochs=100,
                initial_epoch = 5,
                steps_per_epoch= 3803//64,
                validation_data=val_generator,
                validation_steps = 947//64,
                callbacks=[model_checkpoint,early_stopping])

Epoch 6/100
59/59 [==============================] - ETA: 0s - loss: 0.0234 - accuracy: 0.9947
Epoch 6: val_accuracy did not improve from 0.94531
59/59 [==============================] - 74s 1s/step - loss: 0.0234 - accuracy: 0.9947 - val_loss: 0.2426 - val_accuracy: 0.9268
Epoch 7/100
59/59 [==============================] - ETA: 0s - loss: 0.0188 - accuracy: 0.9968
Epoch 7: val_accuracy did not improve from 0.94531
59/59 [==============================] - 72s 1s/step - loss: 0.0188 - accuracy: 0.9968 - val_loss: 0.2238 - val_accuracy: 0.9382
Epoch 8/100
 4/59 [=>............................] - ETA: 41s - loss: 0.0144 - accuracy: 0.9922

KeyboardInterrupt: ignored

###Predicting on Test Set

In [27]:
#Load saved weights
final_model.load_weights('plant_seedlings_resnet50.h5')

In [28]:
#Define Test Data Generator without Augmentation
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function = normalize_data)

In [29]:
test_generator = datagen.flow_from_directory('test',
                                             target_size = (img_size, img_size),
                                             batch_size = 1,
                                             shuffle = False)

Found 794 images belonging to 1 classes.


In [30]:
#Extract predictions
preds_test = final_model.predict(test_generator, verbose=1)

794/794 [==============================] - 16s 19ms/step


In [31]:
#Get argmax for predictions
pred_classes = np.argmax(preds_test, axis = 1)

###Preparing CSV file for submission

In [34]:
#Get filenames from test generator and put them into a dataframe
import pandas as pd
submission = pd.DataFrame()
submission['filenames'] = test_generator.filenames
print(submission.shape)

(794, 1)


In [35]:
#The file names have the folder names as well in them, need to get rid of them
new = submission["filenames"].str.split("/", n = 1, expand = True) 
submission["file"]= new[1] 
submission['species'] = pred_classes
submission = submission.drop(['filenames'],axis=1)
submission.head()

,file,species
0,0021e90e4.png,10
1,003d61042.png,5
2,007b3da8b.png,11
3,0086a6340.png,3
4,00c47e980.png,11


In [36]:
#Change the numerical values in the species column back to the original class names
label_map = (train_generator.class_indices)
label_map

{'Black-grass': 0,
 'Charlock': 1,
 'Cleavers': 2,
 'Common Chickweed': 3,
 'Common wheat': 4,
 'Fat Hen': 5,
 'Loose Silky-bent': 6,
 'Maize': 7,
 'Scentless Mayweed': 8,
 'Shepherds Purse': 9,
 'Small-flowered Cranesbill': 10,
 'Sugar beet': 11}

In [37]:
submission['species'] = submission['species'].replace(0,"Black-grass")
submission['species'] = submission['species'].replace(1,"Charlock")
submission['species'] = submission['species'].replace(2,"Cleavers")
submission['species'] = submission['species'].replace(3,"Common Chickweed")
submission['species'] = submission['species'].replace(4,"Common wheat")
submission['species'] = submission['species'].replace(5,"Fat Hen")
submission['species'] = submission['species'].replace(6,"Loose Silky-bent")
submission['species'] = submission['species'].replace(7,"Maize")
submission['species'] = submission['species'].replace(8,"Scentless Mayweed")
submission['species'] = submission['species'].replace(9,"Shepherds Purse")
submission['species'] = submission['species'].replace(10,"Small-flowered Cranesbill")
submission['species'] = submission['species'].replace(11,"Sugar beet")
submission = submission.set_index('file')
submission.head()

,species
file,
0021e90e4.png,Small-flowered Cranesbill
003d61042.png,Fat Hen
007b3da8b.png,Sugar beet
0086a6340.png,Common Chickweed
00c47e980.png,Sugar beet


In [38]:
#Export the dataframe as a csv file for submission/scoring
submission.to_csv('submission.csv')